In [1]:
# Run command below with docker installed to download and run a spark docker instance:
# docker run --name sparkbook -p 8881:8888 -v "$PWD":/home/jovyan/work jupyter/pyspark-notebook start.sh jupyter lab --LabApp.token=''
# Next, open a web browser and navigate to localhost:8881 to enter docker container. This notebook can then be run.

In [2]:
import sys
!{sys.executable} -m pip install nltk
#Required when running from a newly created spark docker container

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#required for nltk functionality

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark import SparkConf, SparkContext
from sklearn.decomposition import NMF
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import sys
import string
from collections import defaultdict
import numpy as np

In [5]:
#Creates Spark context and session. Sets
conf = SparkConf().setAll([('spark.executor.memory', '15g'), ('spark.driver.memory', '30g')])
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()
print(sc.getConf().getAll())

[('spark.driver.memory', '30g'), ('spark.driver.port', '45157'), ('spark.app.name', 'SimpleApp'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.executor.id', 'driver'), ('spark.submit.deployMode', 'client'), ('spark.app.id', 'local-1562022558122'), ('spark.driver.host', 'a3343a2ac2ad'), ('spark.executor.memory', '15g'), ('spark.ui.showConsoleProgress', 'true')]


In [6]:
#prints current configuration of spark context
print(sc.getConf().getAll())

[('spark.driver.memory', '30g'), ('spark.driver.port', '45157'), ('spark.app.name', 'SimpleApp'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.executor.id', 'driver'), ('spark.submit.deployMode', 'client'), ('spark.app.id', 'local-1562022558122'), ('spark.driver.host', 'a3343a2ac2ad'), ('spark.executor.memory', '15g'), ('spark.ui.showConsoleProgress', 'true')]


In [44]:
#Creates a spark datafrom from the amazon reviews file. Using .limit(x) will subset the first x items.
df = spark.read.csv("data/gaming_reviews.tsv", sep="\t", header=True, inferSchema=True).limit(5000)

In [45]:
df.count()

5000

In [46]:
# Stopwords to filter from  
stop_words = set(stopwords.words('english'))
stop_words.add('')

In [47]:
# removes punctuation, adds column 'body_list' which is a list of words, and selects only the star_rating and body_list columns
stop_words = set(stopwords.words('english'))
stop_words.add('')

def process_str(row):
    """Input: String
    Output: List of strings without punctuation
    
    Removes punctuation using functions from the strings library. Performs raw string operations in C via a lookup table for maximum performance."""
    word_list = row.translate(str.maketrans('', '', string.punctuation)).split(' ')
    return [x.lower() for x in word_list if x.lower() not in stop_words]

process = udf(process_str, ArrayType(StringType()))

df_new = df.withColumn('body_list', process(df['review_body']))\
        .withColumn('body_list', process(df['review_body']))\
        .select('star_rating', 'body_list')
df_new.head()

Row(star_rating=5, body_list=['used', 'elite', 'dangerous', 'mac', 'amazing', 'joystick', 'especially', 'love', 'twist', 'stick', 'different', 'movement', 'bindings', 'well', 'move', 'normal', 'way'])

In [48]:
cv = CountVectorizer(inputCol="body_list", outputCol="features")
idf = IDF(inputCol='features', outputCol = 'idf_features')
rf = RandomForestClassifier(labelCol='star_rating', featuresCol='idf_features', seed=100)

In [49]:
# Stage one in pipeline the words are separated into count vectors
stage1_fit = cv.fit(df_new)
stage1_transform = stage1_fit.transform(df_new)

In [50]:
# Stage two in pipeline for tf-IDF
stage2_fit = idf.fit(stage1_transform)
stage2_transform = stage2_fit.transform(stage1_transform)

In [51]:
# Stage three in pipeline for random forest
stage3_fit = rf.fit(stage2_transform)
stage3_transform = stage3_fit.transform(stage2_transform)

In [52]:
# Function that prints the top n features
def print_top_features(model, n=10):
    """
    Input: RandomForest model, number of top feature words to print.
    Output: None. Prints top n words to console.
    """
    sorted_indices = np.flip(np.argsort(model.featureImportances.toArray()))
    for i in range(20):
        print(stage1_fit.vocabulary[sorted_indices[i]])
print_top_features(stage3_fit);

4
like
return
get
bad
ok
doesnt
sucks
refund
well
one
broke
play
stars
2
xbox
great
isnt
however
never


In [53]:
star_idf = stage3_transform.select('star_rating', 'idf_features')
vocabulary = np.array(stage1_fit.vocabulary)

In [54]:
nmf_model = NMF(n_components=10)

In [55]:
X = [x[0].toArray() for x in star_idf.select('idf_features').collect()] 
nmf_fit = nmf_model.fit(X)
nmf_transform = nmf_model.transform(X)

In [56]:
for i in range(nmf_fit.components_.shape[0]):
    print(vocabulary[np.flip(np.argsort(nmf_fit.components_[i, :]))[0:10]])

['button' 'press' 'consist' 'attack' 'br' 'toy' 'wars' 'force'
 'lightsaber' 'triangle']
['bundle' '1tb' 'bundles' 'includes' 'rating' 'console' 'x1' 'games'
 'include' '399']
['gears' 'war' 'br' 'shotgun' 'crosshairs' 'remember' 'like' 'new' 'next'
 'original']
['br' 'game' 'like' 'get' 'play' 'one' 'would' 'games' 'time' 'dont']
['sonic' '62' 'sly' 'cooper' 'level' 'youll' 'boom' 'levels' 'gorgeous'
 '3d']
['wrestlers' 'created' 'wwe' 'wrestling' 'roster' '2k15' 'theres' 'online'
 'week' 'access']
['music' 'songs' 'br' 'band' 'rock' 'song' 'notes' 'like' 'note' 'guitar']
['units' 'robot' 'wars' 'z' 'series' 'z3' 'z2' 'attack' 'super' 'return']
['backstage' 'wcw' 'assault' 'hardcore' 'match' 'women' 'challenge'
 'opponent' 'wcws' 'matches']
['dishonored' 'thief' 'titlebr' 'cause' 'gun' 'fps' 'get' 'vo' 'ive'
 'bloody']
